In [ ]:
# installing pyarrow and pyspark
# pip install pyarrow
# pip install pyspark delta-spark


In [7]:
# import sys
# print(sys.version)
# from pyspark.sql import SparkSession

# spark = SparkSession.builder \
#     .appName("TestApp") \
#     .getOrCreate()

# print(f"spark version : {spark.version}")
# spark.stop()

from pyspark.sql import SparkSession
from delta import *

# Initialize Spark session with Delta support
spark = SparkSession.builder \
    .appName("DeltaExample") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.2.1") \
    .getOrCreate()


data = [("Alice", 1), ("Bob", 2)]
df = spark.createDataFrame(data, ["name", "id"])

# Test writing to a Delta table
path = "/tmp/delta-table"
df.write.format("delta").mode("overwrite").save(path)

# Test reading from a Delta table
df_read = spark.read.format("delta").load(path)
df_read.show()



Py4JJavaError: An error occurred while calling o151.save.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: delta. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:863)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:257)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:240)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:75)
	at java.base/jdk.inte

In [19]:
# 1. Creating a Sample CSV File

import pandas as pd
import numpy as np

# Creating a DataFrame
data = {
    "id": np.arange(1, 11),
    "name": ["Alice", "Bob", "Charlie", "David", "Eva", "Fiona", "Gina", "Harry", "Ivy", "John"],
    "age": np.random.randint(18, 40, size=10)
}

# write the content of df to CSV file
df = pd.DataFrame(data)
print("df to csv")
print(df)

# Saving to CSV
csv_file_path = 'sample_data.csv'
print(csv_file_path)
df.to_csv(csv_file_path, index=False)
print("\n after csv file creation")
print(df)

df to csv
   id     name  age
0   1    Alice   33
1   2      Bob   18
2   3  Charlie   32
3   4    David   27
4   5      Eva   34
5   6    Fiona   22
6   7     Gina   31
7   8    Harry   36
8   9      Ivy   31
9  10     John   28
sample_data.csv

 after csv file creation
   id     name  age
0   1    Alice   33
1   2      Bob   18
2   3  Charlie   32
3   4    David   27
4   5      Eva   34
5   6    Fiona   22
6   7     Gina   31
7   8    Harry   36
8   9      Ivy   31
9  10     John   28


In [ ]:
# 2. Converting CSV to Parquet

# Reading CSV
df = pd.read_csv(csv_file_path)

# Converting to Parquet
parquet_file_path = 'sample_data.parquet'
df.to_parquet(parquet_file_path, index=False)
print("after converting csv file to parquet file")
print(df)

In [ ]:
# 3. Accessing Parquet Metadata and Viewing Blocks

import pyarrow.parquet as pq

# Opening the Parquet file
parquet_file = pq.ParquetFile(parquet_file_path)

# Viewing Metadata
print(parquet_file.metadata)
print(parquet_file.schema)

# Viewing the data by row group and column
for i in range(parquet_file.num_row_groups):
    rg = parquet_file.read_row_group(i)
    print(f"Row Group {i}:")
    print(rg.to_pandas())


In [24]:
# 4. Converting CSV to Delta-Parquet and Viewing Data

from pyspark.sql import SparkSession
from delta import *

# Initialize Spark session with Delta support
spark = SparkSession.builder \
    .appName("DeltaExample") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Ensure the DataFrame from the CSV is loaded into Spark
df_spark = spark.read.csv('sample_data.csv', header=True, inferSchema=True)
print(df_spark)

# Define a path where the Delta table will be stored
path_to_delta_table = "/tmp/delta-table"

# Write DataFrame to Delta format
df_spark.write.format("delta").mode("overwrite").save(path_to_delta_table)

# Reading from the Delta table
delta_df = spark.read.format("delta").load(path_to_delta_table)
delta_df.show()

# Accessing DeltaTable functions
delta_table = DeltaTable.forPath(spark, path_to_delta_table)
full_history = delta_table.history()  # Provides a DataFrame with the full history of the table
full_history.show()



Py4JJavaError: An error occurred while calling o210.csv.
: org.apache.spark.SparkException: Cannot find catalog plugin class for catalog 'spark_catalog': org.apache.spark.sql.delta.catalog.DeltaCatalog.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.catalogPluginClassNotFoundForCatalogError(QueryExecutionErrors.scala:1925)
	at org.apache.spark.sql.connector.catalog.Catalogs$.load(Catalogs.scala:70)
	at org.apache.spark.sql.connector.catalog.CatalogManager.loadV2SessionCatalog(CatalogManager.scala:67)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$v2SessionCatalog$2(CatalogManager.scala:86)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$v2SessionCatalog$1(CatalogManager.scala:86)
	at scala.Option.map(Option.scala:230)
	at org.apache.spark.sql.connector.catalog.CatalogManager.v2SessionCatalog(CatalogManager.scala:85)
	at org.apache.spark.sql.connector.catalog.CatalogManag

In [ ]:
from pyspark.sql import SparkSession

# Initialize Spark session with Delta support
# spark = SparkSession.builder \
#     .appName("DeltaExample") \
#     .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#     .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
#     .config("spark.jars.packages", "io.delta:delta-core_2.12:1.2.1") \
#     .getOrCreate()

spark = SparkSession.builder \
    .appName("DeltaExample") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

test_df = spark.createDataFrame([("Alice", 1), ("Bob", 2)], ["name", "id"])


print(test_df)